# Using the method developed in the other notebook "*Classifying sources with the WHAV\* diagram*" can we successfully classify spaxels in SAMI galaxies? 
---

In [1]:
%matplotlib widget

In [2]:
# Imports
import sys
import os 
import numpy as np
import pandas as pd
from astropy.visualization import hist

from spaxelsleuth.loaddata.lzifu import load_lzifu_galaxies
from spaxelsleuth.loaddata.sami import load_sami_galaxies
from spaxelsleuth.plotting.plottools import plot_empty_BPT_diagram
from spaxelsleuth.plotting.plottools import vmin_fn, vmax_fn, label_fn, cmap_fn, fname_fn
from spaxelsleuth.plotting.plottools import bpt_colours, bpt_labels, whav_colors, whav_labels
from spaxelsleuth.plotting.plottools import morph_labels, morph_ticks
from spaxelsleuth.plotting.plottools import ncomponents_labels, ncomponents_colours
from spaxelsleuth.plotting.plottools import component_labels, component_colours
from spaxelsleuth.plotting.plotgalaxies import plot2dhistcontours, plot2dscatter, plot2dcontours
from spaxelsleuth.plotting.plot2dmap import plot2dmap
from spaxelsleuth.plotting.sdssimg import plot_sdss_image

import matplotlib
from matplotlib import rc, rcParams
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

rc("text", usetex=False)
rc("font",**{"family": "serif", "size": 14})
rcParams["savefig.bbox"] = "tight"
rcParams["savefig.format"] = "pdf"
plt.ion()
plt.close("all")


In [3]:
# Options
fig_path = "/priv/meggs3/u5708159/SAMI/figs/paper/"
savefigs = True
bin_type = "default"    # Options: "default" or "adaptive" for Voronoi binning
ncomponents = "recom"   # Options: "1" or "recom"
eline_SNR_min = 3       # Minimum S/N of emission lines to accept
plt.close("all")


In [4]:
# Load the sample
df = load_sami_galaxies(ncomponents=ncomponents,
                        bin_type=bin_type,
                        eline_SNR_min=eline_SNR_min, 
                        vgrad_cut=False,
                        correct_extinction=False,
                        sigma_gas_SNR_cut=True)

/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:853: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc,

In [5]:
# Load the LZIFU galaxies
df_lzifu = load_lzifu_galaxies(ncomponents=ncomponents,
                              bin_type=bin_type,
                              eline_SNR_min=5, 
                              vgrad_cut=False,
                              correct_extinction=False,
                              sigma_gas_SNR_cut=True)        


Loading LZIFU DataFrame for all galaxies in the LZIFU subsample...


/home/u5708159/python/Modules/spaxelsleuth/loaddata/linefns.py:78: RuntimeWarning: invalid value encountered in add
  return -0.943 * ratio_y_vals**4 - 0.450 * ratio_y_vals**3 + 0.408 * ratio_y_vals**2 - 0.610 * ratio_y_vals - 0.025


## Plots for paper: histograms showing log N2, EW, $\Delta\sigma$ for each BPT category separately
---

Histograms of each quantity, separated by BPT category, for 
* LZIFU subset (individual components)
* SAMI dataset (based on total line fluxes)

In [6]:
plt.close("all")

In [7]:
#//////////////////////////////////////////////////////////////////////////////
# Make a copy of the LZIFU data frame where the fluxes/classifications for 
# all components are stored in "component 0"
df_lzifu_comp0 = None
for ii in [0, 1, 2]:
    df_this_component = df_lzifu[~df_lzifu[f"sigma_gas - sigma_* (component {ii})"].isna()]
    # # print(df_this_component[print_cols])
    
    # Drop all columns that are NOT this component
    other_components = [cc for cc in [0, 1, 2] if cc != ii]
    cols_to_drop = [c for c in df_this_component.columns if f"(total)" in c]
    for cc in other_components:
        cols_to_drop += [c for c in df_this_component.columns if f"(component {cc})" in c]
    df_this_component = df_this_component.drop(columns=cols_to_drop)
    # print("---------------------------------------------------")
    # print(df_this_component[f"sigma_gas - sigma_* (component {ii})"])

    # Rename columns to have suffix "component 0"
    if ii != 0:
        cols_to_rename = [c for c in df_this_component.columns if c.endswith(f"(component {ii})")]
        new_col_names = [c.split(f"(component {ii})")[0] + "(component 0)" for c in cols_to_rename]
        rename_dict = dict(zip(cols_to_rename, new_col_names))
        df_this_component = df_this_component.rename(columns=rename_dict)
    # print("---------------------------------------------------")
    # print(df_this_component[f"sigma_gas - sigma_* (component 0)"])

    # Merge 
    if df_lzifu_comp0 is None:
        df_lzifu_comp0 = df_this_component.copy()
    else:
        df_lzifu_comp0 = df_lzifu_comp0.append(df_this_component)
    # print("---------------------------------------------------")
    # print(df_lzifu_comp0[f"sigma_gas - sigma_* (component 0)"])

# Drop bad rows
df_lzifu_comp0.loc[:, "Good?"] = ~df_lzifu_comp0[f"sigma_gas - sigma_* (component 0)"].isna() & ~df_lzifu_comp0[f"log HALPHA EW (component 0)"].isna()
cond = df_lzifu_comp0["Good?"] == 1.0
df_lzifu_comp0 = df_lzifu_comp0[cond]


/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [8]:
# Histograms showing Halpha EW as a function of spectral category
col = "log HALPHA EW (component 0)"
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8, 5))
for cat, colour in zip(bpt_labels, bpt_colours):
    df_cat = df_lzifu_comp0[df_lzifu_comp0["BPT (component 0)"] == cat]
    N = df_cat.loc[~np.isnan(df_cat[col])].shape[0]
    hist(df_cat[col], range=(vmin_fn(col), vmax_fn(col)), bins="scott", histtype="step", color=colour, label=f"{cat} (N = {N:d})", normed=True)
ax.set_xlabel(label_fn(col))
ax.set_ylabel(r"$N$ (normalised)")
ax.set_title("Individual components")
ax.grid()
ax.legend(fontsize="x-small")
ax.autoscale(axis="x", tight=True, enable=True)
if savefigs:
    fig.savefig(os.path.join(fig_path, f"lzifu_subset_hist_{fname_fn(col)}_indv.pdf"), format="pdf", bbox_inches="tight")

    # Histograms showing Halpha EW as a function of spectral category
col = "log HALPHA EW (total)"
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8, 5))
for cat, colour in zip(bpt_labels, bpt_colours):
    df_cat = df_lzifu[df_lzifu["BPT (total)"] == cat]
    N = df_cat.loc[~np.isnan(df_cat[col])].shape[0]
    hist(df_cat[col], range=(vmin_fn(col), vmax_fn(col)), bins="scott", histtype="step", color=colour, label=f"{cat} (N = {N:d})", normed=True)
ax.set_xlabel(label_fn(col))
ax.set_ylabel(r"$N$ (normalised)")
ax.set_title("Spaxels")
ax.grid()
ax.legend(fontsize="x-small")
ax.autoscale(axis="x", tight=True, enable=True)
if savefigs:
    fig.savefig(os.path.join(fig_path, f"lzifu_subset_hist_{fname_fn(col)}_total.pdf"), format="pdf", bbox_inches="tight")

    
# Histograms showing delta sigma as a function of spectral category
col = "sigma_gas - sigma_* (component 0)"
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8, 5))
for cat, colour in zip(bpt_labels, bpt_colours):
    df_cat = df_lzifu_comp0[df_lzifu_comp0["BPT (component 0)"] == cat]
    N = df_cat.loc[~np.isnan(df_cat[col])].shape[0]
    hist(df_cat[col], range=(vmin_fn(col), 400), bins="scott", histtype="step", color=colour, label=f"{cat} (N = {N:d})", normed=True)
ax.set_xlabel(label_fn(col))
ax.set_ylabel(r"$N$ (normalised)")
ax.set_title("Individual components")
ax.grid()
ax.legend(fontsize="x-small")
ax.autoscale(axis="x", tight=True, enable=True)
if savefigs:
    fig.savefig(os.path.join(fig_path, f"lzifu_subset_hist_{fname_fn(col)}.pdf"), format="pdf", bbox_inches="tight")


# Histograms showing log N2 as a function of spectral category - INDIVIDUAL COMPONENTS
col = "log N2 (component 0)"
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8, 5))
for cat, colour in zip(bpt_labels, bpt_colours):
    df_cat = df_lzifu_comp0[df_lzifu_comp0["BPT (component 0)"] == cat]
    N = df_cat.loc[~np.isnan(df_cat[col])].shape[0]
    hist(df_cat[col], range=(vmin_fn(col), vmax_fn(col)), bins="scott", histtype="step", color=colour, label=f"{cat} (N = {N:d})", normed=True)
ax.axvline(-0.35, color="k", linestyle="--")
ax.axvline(-0.20, color="k", linestyle="--")
ax.set_xlabel(label_fn(col))
ax.set_ylabel(r"$N$ (normalised)")
ax.set_title("Individual components")
ax.grid()
ax.legend(fontsize="x-small")
ax.autoscale(axis="x", tight=True, enable=True)
if savefigs:
    fig.savefig(os.path.join(fig_path, f"lzifu_subset_hist_{fname_fn(col)}_indv.pdf"), format="pdf", bbox_inches="tight")


# Histograms showing log N2 as a function of spectral category - SPAXELS
col = "log N2 (total)"
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8, 5))
for cat, colour in zip(bpt_labels, bpt_colours):
    df_cat = df_lzifu[df_lzifu["BPT (total)"] == cat]
    N = df_cat.loc[~np.isnan(df_cat[col])].shape[0]
    hist(df_cat[col], range=(vmin_fn(col), vmax_fn(col)), bins="scott", histtype="step", color=colour, label=f"{cat} (N = {N:d})", normed=True)
ax.axvline(-0.35, color="k", linestyle="--")
ax.axvline(-0.20, color="k", linestyle="--")
ax.set_xlabel(label_fn(col))
ax.set_ylabel(r"$N$ (normalised)")
ax.set_title("Spaxels")
ax.grid()
ax.legend(fontsize="x-small")
ax.autoscale(axis="x", tight=True, enable=True)
if savefigs:
    fig.savefig(os.path.join(fig_path, f"lzifu_subset_hist_{fname_fn(col)}_total.pdf"), format="pdf", bbox_inches="tight")



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/astropy/visualization/hist.py:72: MatplotlibDeprecationWarning: 
The 'normed' kwarg was deprecated in Matplotlib 2.1 and will be removed in 3.1. Use 'density' instead.
  return ax.hist(x, bins, **kwargs)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/astropy/visualization/hist.py:72: MatplotlibDeprecationWarning: 
The 'normed' kwarg was deprecated in Matplotlib 2.1 and will be removed in 3.1. Use 'density' instead.
  return ax.hist(x, bins, **kwargs)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/astropy/visualization/hist.py:72: MatplotlibDeprecationWarning: 
The 'normed' kwarg was deprecated in Matplotlib 2.1 and will be removed in 3.1. Use 'density' instead.
  return ax.hist(x, bins, **kwargs)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/astropy/visualization/hist.py:72: MatplotlibDeprecationWarning: 
The 'normed' kwarg was deprecated in Matplotlib 2.1 and will be removed in 3.1. Use 'density' instead.
  return ax.hist(x, bins, **kwargs)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/astropy/visualization/hist.py:72: MatplotlibDeprecationWarning: 
The 'normed' kwarg was deprecated in Matplotlib 2.1 and will be removed in 3.1. Use 'density' instead.
  return ax.hist(x, bins, **kwargs)


## Tables: comparison between classification systems
--- 

In [9]:
"""
################################################################################
# Testing our classifiction system
################################################################################
df_lzifu["WHAV*"] = "Unknown"  # Initialise everything to "unknown"

#///////////////////////////////////////////////////////////////////////////////
# Step 1: filter out evolved stars
cond = df_lzifu["HALPHA EW (total)"] <= 3
df_lzifu.loc[cond, "WHAV*"] = "HOLMES"
cond_remainder = df_lzifu["WHAV*"] == "Unknown"

#///////////////////////////////////////////////////////////////////////////////
# Step 2: use the N2 ratio to divide into SF, mixed and AGN/evolved stars/shocks
# Because we used the TOTAL N2 ratio in each spaxel to determine these boundaries, these categories are representative of the DOMINANT ionisation mechanism in each spaxel.
cond_SF = cond_remainder & (df_lzifu["log N2 (total)"] < -0.35)
cond_Mixing = cond_remainder & (df_lzifu["log N2 (total)"] >= -0.35) & (df_lzifu["log N2 (total)"] < -0.2)
cond_AGN = cond_remainder & (df_lzifu["log N2 (total)"] >= -0.2)
cond_no_N2 = np.isnan(df_lzifu["log N2 (total)"])

df_lzifu.loc[cond_SF, "WHAV*"] = "SF" 
df_lzifu.loc[cond_Mixing, "WHAV*"] = "Mixing"
df_lzifu.loc[cond_AGN, "WHAV*"] = "AGN/HOLMES/shocks"
df_lzifu.loc[cond_no_N2, "WHAV*"] = "No N2"

#///////////////////////////////////////////////////////////////////////////////
# For convenience: mark components as possible HOLMES 
# Question: how confident can we be that these are ALWAYS HOLMES? how common are components from e.g. LLAGN?
for ii in range(3):
    cond_possible_HOLMES = cond_AGN & (df_lzifu[f"HALPHA EW (component {ii})"] < 3) & (df_lzifu[f"sigma_gas - sigma_* (component {ii})"] < 0)
    df_lzifu.loc[cond_possible_HOLMES, f"Possible HOLMES (component {ii})"] = True
    df_lzifu.loc[~cond_possible_HOLMES, f"Possible HOLMES (component {ii})"] = False
    
#///////////////////////////////////////////////////////////////////////////////
# For convenience: mark components as being kinematically disturbed (by 3sigma)
for ii in range(3):
    cond_kinematically_disturbed = df_lzifu[f"sigma_gas - sigma_* (component {ii})"] - 3 * df_lzifu[f"sigma_gas - sigma_* error (component {ii})"] > 0
    df_lzifu.loc[cond_kinematically_disturbed, f"Kinematically disturbed (component {ii})"] = True
    df_lzifu.loc[~cond_kinematically_disturbed, f"Kinematically disturbed (component {ii})"] = False
    
################################################################################
# Repeat for SAMI sample
################################################################################
df["WHAV*"] = "Unknown"  # Initialise everything to "unknown"

#///////////////////////////////////////////////////////////////////////////////
# Step 1: filter out evolved stars
cond = df["HALPHA EW (total)"] <= 3
df.loc[cond, "WHAV*"] = "HOLMES"
cond_remainder = df["WHAV*"] == "Unknown"

#///////////////////////////////////////////////////////////////////////////////
# Step 2: use the N2 ratio to divide into SF, mixed and AGN/evolved stars/shocks
# Because we used the TOTAL N2 ratio in each spaxel to determine these boundaries, these categories are representative of the DOMINANT ionisation mechanism in each spaxel.
cond_SF = cond_remainder & (df["log N2 (total)"] < -0.35)
cond_Mixing = cond_remainder & (df["log N2 (total)"] >= -0.35) & (df["log N2 (total)"] < -0.2)
cond_AGN = cond_remainder & (df["log N2 (total)"] >= -0.2)
cond_no_N2 = np.isnan(df["log N2 (total)"])

df.loc[cond_SF, "WHAV*"] = "SF" 
df.loc[cond_Mixing, "WHAV*"] = "Mixing"
df.loc[cond_AGN, "WHAV*"] = "AGN/HOLMES/shocks"
df.loc[cond_no_N2, "WHAV*"] = "No N2"

#///////////////////////////////////////////////////////////////////////////////
# For convenience: mark components as possible HOLMES 
# Question: how confident can we be that these are ALWAYS HOLMES? how common are components from e.g. LLAGN?
for ii in range(3):
    cond_possible_HOLMES = cond_AGN & (df[f"HALPHA EW (component {ii})"] < 3) & (df[f"sigma_gas - sigma_* (component {ii})"] < 0)
    df.loc[cond_possible_HOLMES, f"Possible HOLMES (component {ii})"] = True
    df.loc[~cond_possible_HOLMES, f"Possible HOLMES (component {ii})"] = False
    
#///////////////////////////////////////////////////////////////////////////////
# For convenience: mark components as being kinematically disturbed (by 3sigma)
for ii in range(3):
    cond_kinematically_disturbed = df[f"sigma_gas - sigma_* (component {ii})"] - 3 * df[f"sigma_gas - sigma_* error (component {ii})"] > 0
    df.loc[cond_kinematically_disturbed, f"Kinematically disturbed (component {ii})"] = True
    df.loc[~cond_kinematically_disturbed, f"Kinematically disturbed (component {ii})"] = False
"""

'\n################################################################################\n# Testing our classifiction system\n################################################################################\ndf_lzifu["WHAV*"] = "Unknown"  # Initialise everything to "unknown"\n\n#///////////////////////////////////////////////////////////////////////////////\n# Step 1: filter out evolved stars\ncond = df_lzifu["HALPHA EW (total)"] <= 3\ndf_lzifu.loc[cond, "WHAV*"] = "HOLMES"\ncond_remainder = df_lzifu["WHAV*"] == "Unknown"\n\n#///////////////////////////////////////////////////////////////////////////////\n# Step 2: use the N2 ratio to divide into SF, mixed and AGN/evolved stars/shocks\n# Because we used the TOTAL N2 ratio in each spaxel to determine these boundaries, these categories are representative of the DOMINANT ionisation mechanism in each spaxel.\ncond_SF = cond_remainder & (df_lzifu["log N2 (total)"] < -0.35)\ncond_Mixing = cond_remainder & (df_lzifu["log N2 (total)"] >= -0.35) & (

In [10]:
df_lzifu["WHAV*"].unique()

array(['Unknown', 'SF + no wind', 'Mixing + no wind', 'AGN + no wind',
       'SF + wind', 'HOLMES', 'Mixing + wind', 'AGN + wind',
       'AGN + HOLMES + no wind', 'AGN + HOLMES + wind'], dtype=object)

In [11]:
#############################################################################################################
# How many spaxels can each method successfully classify?
#############################################################################################################
print("In the high-S/N LZIFU subset:")
cond_has_emission_lines = df_lzifu["Number of components"] >= 1
frac_unclassified_bpt = 1 - df_lzifu[cond_has_emission_lines & (df_lzifu["BPT (total)"] == "Not classified")].shape[0] / df_lzifu[cond_has_emission_lines].shape[0]
# frac_unclassified_whan = 1 - df_lzifu[cond_has_emission_lines & (df_lzifu["WHAN"] == "No N2")].shape[0] / df_lzifu[cond_has_emission_lines].shape[0]
frac_unclassified_whav = 1 - df_lzifu[cond_has_emission_lines & (df_lzifu["WHAV*"] == "Unknown")].shape[0] / df_lzifu[cond_has_emission_lines].shape[0]
print(f"{frac_unclassified_bpt * 100}\% of spaxels can be classified using the BPT")
# print(f"{frac_unclassified_whan * 100}\% of spaxels can be classified using the WHAN")
print(f"{frac_unclassified_whav * 100}\% of spaxels can be classified using the new method")

print("\nIn the whole SAMI sample:")
cond_has_emission_lines = df["Number of components"] >= 1
frac_unclassified_bpt = 1 - df[cond_has_emission_lines & (df["BPT (total)"] == "Not classified")].shape[0] / df[cond_has_emission_lines].shape[0]
# frac_unclassified_whan = 1 - df[cond_has_emission_lines & (df["WHAN"] == "No N2")].shape[0] / df[cond_has_emission_lines].shape[0]
frac_unclassified_whav = 1 - df[cond_has_emission_lines & (df["WHAV*"] == "Unknown")].shape[0] / df[cond_has_emission_lines].shape[0]
print(f"{frac_unclassified_bpt * 100}\% of spaxels can be classified using the BPT")
# print(f"{frac_unclassified_whan * 100}\% of spaxels can be classified using the WHAN")
print(f"{frac_unclassified_whav * 100}\% of spaxels can be classified using the new method")

In the high-S/N LZIFU subset:
31.76630286387431\% of spaxels can be classified using the BPT
90.15704110529478\% of spaxels can be classified using the new method

In the whole SAMI sample:
51.750623984113055\% of spaxels can be classified using the BPT
89.72571902692484\% of spaxels can be classified using the new method


In [12]:
#############################################################################################################
# Table: comparison between BPT- and N2- and \ewha{}-based spectral categories for the LZIFU high-SN subset.
#############################################################################################################
s = f"{'':20s}"
s_tot = f"{'Total':20s}"
for bpt in ["SF", "Composite", "Seyfert", "LINER", "Ambiguous", "Not classified"]:
    s += f"& {bpt:15s}"
    N_bpt_tot =         df_lzifu[(df_lzifu["BPT (total)"] == bpt)].shape[0]
    s_tot += f"& {N_bpt_tot:15d} "
s += "\\\\"
s_tot += "\\\\"
print(s)
for whav_cat in ["SF", "Mixing", "AGN/HOLMES/shocks", "HOLMES", "No N2", "Unknown"]:
    s = f"{whav_cat:20s}"
    for bpt in ["SF", "Composite", "Seyfert", "LINER", "Ambiguous", "Not classified"]:
        N_bpt_tot =         df_lzifu[(df_lzifu["BPT (total)"] == bpt)].shape[0]
        N_bpt_in_this_cat = df_lzifu[(df_lzifu["BPT (total)"] == bpt) & (df_lzifu["WHAN"] == whav_cat)].shape[0]
        s += f"\t& {N_bpt_in_this_cat / N_bpt_tot * 100:10.2f}\%"
    s += "\\\\"
    print(s)
print(s_tot)


                    & SF             & Composite      & Seyfert        & LINER          & Ambiguous      & Not classified \\
SF                  	&      94.13\%	&      21.08\%	&       1.79\%	&       0.00\%	&      70.78\%	&       4.92\%\\
Mixing              	&       5.83\%	&      65.57\%	&       5.46\%	&       0.20\%	&       8.68\%	&       3.92\%\\
AGN/HOLMES/shocks   	&       0.01\%	&      10.13\%	&      66.32\%	&      27.21\%	&       9.99\%	&       1.74\%\\
HOLMES              	&       0.03\%	&       3.21\%	&      26.43\%	&      72.59\%	&      10.54\%	&       5.58\%\\
No N2               	&       0.00\%	&       0.00\%	&       0.00\%	&       0.00\%	&       0.00\%	&      83.84\%\\
Unknown             	&       0.00\%	&       0.00\%	&       0.00\%	&       0.00\%	&       0.00\%	&       0.00\%\\
Total               &           27339 &            7404 &            1063 &            1470 &            3443 &          463155 \\


In [13]:
#############################################################################################################
# Table: comparison between BPT- and N2- and \ewha{}-based spectral categories for the entire SAMI sample.
#############################################################################################################
s = f"{'':20s}"
s_tot = f"{'Total':20s}"
for bpt in ["SF", "Composite", "Seyfert", "LINER", "Ambiguous", "Not classified"]:
    s += f"& {bpt:15s}"
    N_bpt_tot =         df[(df["BPT (total)"] == bpt)].shape[0]
    s_tot += f"& {N_bpt_tot:15d} "
s += "\\\\"
s_tot += "\\\\"
print(s)
for whav_cat in ["SF", "Mixing", "AGN/HOLMES/shocks", "HOLMES", "No N2", "Unknown"]:
    s = f"{whav_cat:20s}"
    for bpt in ["SF", "Composite", "Seyfert", "LINER", "Ambiguous", "Not classified"]:
        N_bpt_tot =         df[(df["BPT (total)"] == bpt)].shape[0]
        N_bpt_in_this_cat = df[(df["BPT (total)"] == bpt) & (df["WHAN"] == whav_cat)].shape[0]
        s += f"\t& {N_bpt_in_this_cat / N_bpt_tot * 100:10.2f}\%"
    s += "\\\\"
    print(s)
print(s_tot)


                    & SF             & Composite      & Seyfert        & LINER          & Ambiguous      & Not classified \\
SF                  	&      96.96\%	&      15.35\%	&       7.22\%	&       0.01\%	&      79.62\%	&       7.47\%\\
Mixing              	&       3.02\%	&      66.29\%	&      15.99\%	&       1.28\%	&       5.92\%	&       3.61\%\\
AGN/HOLMES/shocks   	&       0.00\%	&      11.97\%	&      56.87\%	&      21.36\%	&       5.64\%	&       1.67\%\\
HOLMES              	&       0.02\%	&       6.39\%	&      19.92\%	&      77.35\%	&       8.82\%	&       6.38\%\\
No N2               	&       0.00\%	&       0.00\%	&       0.00\%	&       0.00\%	&       0.00\%	&      80.87\%\\
Unknown             	&       0.00\%	&       0.00\%	&       0.00\%	&       0.00\%	&       0.00\%	&       0.00\%\\
Total               &          301858 &           35284 &            3353 &            8919 &           31054 &         1459687 \\


In [14]:
################################################################################
# Test: how many spaxels get mis-classified using our N2 criterion?
################################################################################
# What % of all BPT-classified SF spaxels are in a WHAN category that includes SF?
print("\n############################################################################################")
print("LZIFU SUBSET: STAR-FORMING SPAXELS")
print("############################################################################################")
N_BPT = df_lzifu[(df_lzifu["BPT (total)"] == "SF")].shape[0]
N_agree = df_lzifu[(df_lzifu["BPT (total)"] == "SF") & (df_lzifu["WHAN"] == "SF")].shape[0]
N_disagree = df_lzifu[(df_lzifu["BPT (total)"] == "SF") & (df_lzifu["WHAN"] != "SF")].shape[0]
print(f"Fraction of BPT-classified SF spaxels with a WHAN classification that agrees: {N_agree}/{N_BPT} = {N_agree / N_BPT * 100:.2f}%")
print(f"Fraction of BPT-classified SF spaxels with a WHAN classification that disagrees: {N_disagree}/{N_BPT} = {N_disagree / N_BPT * 100:.2f}%")
print("--------------------------------------------------------------------------------------------")
print("Of all BPT-classified SF spaxels:")
N_tot = 0
for cat in ["Unknown", "HOLMES", "SF", "Mixing", "AGN/HOLMES/shocks", "No N2"]:
    N_in_this_cat = df_lzifu[(df_lzifu["BPT (total)"] == "SF") & (df_lzifu["WHAN"] == cat)].shape[0]
    print(f"\t{N_in_this_cat:4d}/{N_BPT:4d} = {N_in_this_cat / N_BPT * 100:3.2f}% are in category '{cat}'")
    N_tot += N_in_this_cat

print("\n############################################################################################")
print("LZIFU SUBSET: COMPOSITE SPAXELS")
print("############################################################################################")
N_BPT = df_lzifu[(df_lzifu["BPT (total)"] == "Composite")].shape[0]
N_agree = df_lzifu[(df_lzifu["BPT (total)"] == "Composite") & (df_lzifu["WHAN"] == "Mixing")].shape[0]
N_disagree = df_lzifu[(df_lzifu["BPT (total)"] == "Composite") & (df_lzifu["WHAN"] != "Mixing")].shape[0]
print(f"Fraction of BPT-classified SF spaxels with a WHAN classification that agrees: {N_agree}/{N_BPT} = {N_agree / N_BPT * 100:.2f}%")
print(f"Fraction of BPT-classified SF spaxels with a WHAN classification that disagrees: {N_disagree}/{N_BPT} = {N_disagree / N_BPT * 100:.2f}%")
print("--------------------------------------------------------------------------------------------")
print("Of all BPT-classified Composite spaxels:")
N_tot = 0
for cat in ["Unknown", "HOLMES", "SF", "Mixing", "AGN/HOLMES/shocks", "No N2"]:
    N_in_this_cat = df_lzifu[(df_lzifu["BPT (total)"] == "Composite") & (df_lzifu["WHAN"] == cat)].shape[0]
    print(f"\t{N_in_this_cat:4d}/{N_BPT:4d} = {N_in_this_cat / N_BPT * 100:3.2f}% are in category '{cat}'")
    N_tot += N_in_this_cat

print("\n############################################################################################")
print("LZIFU SUBSET: SEYFERT SPAXELS")
print("############################################################################################")
N_BPT = df_lzifu[(df_lzifu["BPT (total)"] == "Seyfert")].shape[0]
N_agree = df_lzifu[(df_lzifu["BPT (total)"] == "Seyfert") & (df_lzifu["WHAN"] == "AGN/HOLMES/shocks")].shape[0]
N_disagree = df_lzifu[(df_lzifu["BPT (total)"] == "Seyfert") & (df_lzifu["WHAN"] != "AGN/HOLMES/shocks")].shape[0]
print(f"Fraction of BPT-classified SF spaxels with a WHAN classification that agrees: {N_agree}/{N_BPT} = {N_agree / N_BPT * 100:.2f}%")
print(f"Fraction of BPT-classified SF spaxels with a WHAN classification that disagrees: {N_disagree}/{N_BPT} = {N_disagree / N_BPT * 100:.2f}%")
print("--------------------------------------------------------------------------------------------")
print("Of all BPT-classified Seyfert spaxels:")
N_tot = 0
for cat in ["Unknown", "HOLMES", "SF", "Mixing", "AGN/HOLMES/shocks", "No N2"]:
    N_in_this_cat = df_lzifu[(df_lzifu["BPT (total)"] == "Seyfert") & (df_lzifu["WHAN"] == cat)].shape[0]
    print(f"\t{N_in_this_cat:4d}/{N_BPT:4d} = {N_in_this_cat / N_BPT * 100:3.2f}% are in category '{cat}'")
    N_tot += N_in_this_cat

print("\n############################################################################################")
print("LZIFU SUBSET: LINER SPAXELS")
print("############################################################################################")
N_BPT = df_lzifu[(df_lzifu["BPT (total)"] == "LINER")].shape[0]
N_agree = df_lzifu[(df_lzifu["BPT (total)"] == "LINER") & (df_lzifu["WHAN"] != "HOLMES") & (df_lzifu["WHAN"] == "AGN/HOLMES/shocks")].shape[0]
N_disagree = df_lzifu[(df_lzifu["BPT (total)"] == "LINER") & (df_lzifu["WHAN"] != "HOLMES") & (df_lzifu["WHAN"] != "AGN/HOLMES/shocks")].shape[0]
print(f"Fraction of BPT-classified SF spaxels with a WHAN classification that agrees: {N_agree}/{N_BPT} = {N_agree / N_BPT * 100:.2f}%")
print(f"Fraction of BPT-classified SF spaxels with a WHAN classification that disagrees: {N_disagree}/{N_BPT} = {N_disagree / N_BPT * 100:.2f}%")
print("--------------------------------------------------------------------------------------------")
print("Of all BPT-classified LINER spaxels:")
N_tot = 0
for cat in ["Unknown", "HOLMES", "SF", "Mixing", "AGN/HOLMES/shocks", "No N2"]:
    N_in_this_cat = df_lzifu[(df_lzifu["BPT (total)"] == "LINER") & (df_lzifu["WHAN"] == cat)].shape[0]
    print(f"\t{N_in_this_cat:4d}/{N_BPT:4d} = {N_in_this_cat / N_BPT * 100:3.2f}% are in category '{cat}'")
    N_tot += N_in_this_cat


############################################################################################
LZIFU SUBSET: STAR-FORMING SPAXELS
############################################################################################
Fraction of BPT-classified SF spaxels with a WHAN classification that agrees: 25733/27339 = 94.13%
Fraction of BPT-classified SF spaxels with a WHAN classification that disagrees: 1606/27339 = 5.87%
--------------------------------------------------------------------------------------------
Of all BPT-classified SF spaxels:
	   0/27339 = 0.00% are in category 'Unknown'
	   9/27339 = 0.03% are in category 'HOLMES'
	25733/27339 = 94.13% are in category 'SF'
	1595/27339 = 5.83% are in category 'Mixing'
	   2/27339 = 0.01% are in category 'AGN/HOLMES/shocks'
	   0/27339 = 0.00% are in category 'No N2'

############################################################################################
LZIFU SUBSET: COMPOSITE SPAXELS
###############################################

## Plots: BPT, WHAN, WHAV* for each of the BPT categories
---

In [ ]:
#///////////////////////////////////////////////////////////////////////////////
# WHAV* - each component shown separately
col_x = "sigma_gas - sigma_*"
col_y = "log HALPHA EW"
col_z = "count"

for cat in bpt_labels:
    df_cat = df_lzifu[df_lzifu["BPT (total)"] == cat]
    if df_cat.shape[0] == 0:
        continue
        
    fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(15, 4))
    fig.subplots_adjust(wspace=0)
    bbox = axs[-1].get_position()
    fig.add_axes([bbox.x0 + bbox.width, bbox.y0, bbox.width * 0.1, bbox.height])
    
    for ii in range(3):
        if all(df_cat[f"{col_x} (component {ii})"].isna()) or all(df_cat[f"{col_y} (component {ii})"].isna()):
            continue
        plot2dhistcontours(df_cat, 
                           col_x=f"{col_x} (component {ii})", 
                           col_y=f"{col_y} (component {ii})",
                           col_z=col_z, log_z=True if col_z == "count" else False, 
                           ax=axs[ii], nbins=100, vmin=1, vmax=1e3, contours=True, colors="white", 
                           plot_colorbar=True if ii == 2 else False)

    # Decorations
    axs[1].set_title(cat)
    [ax.set_ylabel("") for ax in axs[1:]]
    [ax.set_yticklabels([]) for ax in axs[1:]]

    # Grid on
    [ax.grid() for ax in axs]
    
    # Save 
    if savefigs:
        fig.savefig(os.path.join(fig_path, f"BPT_SAMI_indv_{cat.replace(' ', '_')}_{fname_fn(col_z)}.pdf"), bbox_inches="tight", format="pdf")
        

## Plots: BPT, WHAN, WHAV* for each of our N2 categories
---

In [24]:
col_z = "count"

#///////////////////////////////////////////////////////////////////////////////
# BPT - based on TOTAL fluxes
for cat in ["SF", "Mixing", "AGN/HOLMES/shocks", "HOLMES"]:
    df_cat = df[df["WHAN"] == cat]
    if df_cat.shape[0] == 0:
        continue
    col_y = "log O3 (total)"
    fig, axs, cax = plot_empty_BPT_diagram(colorbar=True, nrows=1, include_Law2021=True)
    
    # Plot 2D histograms of the subset
    plot2dhistcontours(df_cat, col_x="log N2 (total)", col_y=col_y, col_z=col_z, log_z=True if col_z == "count" else False, vmin=1, vmax=1e3, ax=axs[0], nbins=100, contours=True, colors="white", plot_colorbar=False)
    plot2dhistcontours(df_cat, col_x="log S2 (total)", col_y=col_y, col_z=col_z, log_z=True if col_z == "count" else False, vmin=1, vmax=1e3, ax=axs[1], nbins=100, contours=True, colors="white", plot_colorbar=False)
    plot2dhistcontours(df_cat, col_x="log O1 (total)", col_y=col_y, col_z=col_z, log_z=True if col_z == "count" else False, vmin=1, vmax=1e3, ax=axs[2], nbins=100, contours=True, colors="white", cax=cax, plot_colorbar=True)

    # Decorations
    axs[1].set_title(cat)
    [ax.set_ylabel("") for ax in axs[1:]]

    # Grid on
    [ax.grid() for ax in axs]
    


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/matplotlib/colors.py:1110: RuntimeWarning: invalid value encountered in less_equal
  mask |= resdat <= 0


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/matplotlib/colors.py:1110: RuntimeWarning: invalid value encountered in less_equal
  mask |= resdat <= 0


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/matplotlib/colors.py:1110: RuntimeWarning: invalid value encountered in less_equal
  mask |= resdat <= 0


FileNotFoundError: [Errno 2] No such file or directory: '/priv/meggs3/u5708159/SAMI/figs/paper/BPT_SAMI_indv_AGN/HOLMES/shocks_count.pdf'

In [17]:
#///////////////////////////////////////////////////////////////////////////////
# WHAN - each component shown separately
col_x = "log N2"
col_y = "log HALPHA EW"
col_z = "count"

for cat in ["HOLMES", "SF", "Mixing", "AGN/HOLMES/shocks"]:
    df_cat = df_lzifu[df_lzifu["WHAN"] == cat]
    if df_cat.shape[0] == 0:
        continue
        
    fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(15, 4))
    fig.subplots_adjust(wspace=0)
    bbox = axs[-1].get_position()
    fig.add_axes([bbox.x0 + bbox.width, bbox.y0, bbox.width * 0.1, bbox.height])
    
    for ii in range(3):
        if all(df_cat[f"{col_x} (component {ii})"].isna()) or all(df_cat[f"{col_y} (component {ii})"].isna()):
            continue
        plot2dhistcontours(df_cat, 
                           col_x=f"{col_x} (component {ii})", 
                           col_y=f"{col_y} (component {ii})",
                           col_z=col_z, log_z=True if col_z == "count" else False, 
                           ax=axs[ii], nbins=100, vmin=1, vmax=1e3, contours=True, colors="white", 
                           plot_colorbar=True if ii == 2 else False)

    # Decorations
    axs[1].set_title(cat)
    [ax.set_ylabel("") for ax in axs[1:]]
    [ax.set_yticklabels([]) for ax in axs[1:]]

    # Grid on
    [ax.grid() for ax in axs]

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/u5708159/python/Modules/spaxelsleuth/plotting/plotgalaxies.py:178: UserWarning: No contour levels were found within the data range.
  linewidths=linewidths)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/u5708159/python/Modules/spaxelsleuth/plotting/plotgalaxies.py:214: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  cax = fig.add_axes([bbox.x0 + bbox.width, bbox.y0, bbox.width * 0.1, bbox.height])


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
#///////////////////////////////////////////////////////////////////////////////
# WHAV* - each component shown separately
col_x = "sigma_gas - sigma_*"
col_y = "log HALPHA EW"
col_z = "count"

# for cat in ["HOLMES", "SF", "Mixing", "AGN/HOLMES/shocks"]:
for cat in ["No N2"]:
    df_cat = df_lzifu[df_lzifu["WHAN"] == cat]
    if df_cat.shape[0] == 0:
        continue
        
    fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(15, 4))
    fig.subplots_adjust(wspace=0)
    bbox = axs[-1].get_position()
    fig.add_axes([bbox.x0 + bbox.width, bbox.y0, bbox.width * 0.1, bbox.height])
    
    for ii in range(3):
        if all(df_cat[f"{col_x} (component {ii})"].isna()) or all(df_cat[f"{col_y} (component {ii})"].isna()):
            continue
        plot2dhistcontours(df_cat, 
                           col_x=f"{col_x} (component {ii})", 
                           col_y=f"{col_y} (component {ii})",
                           col_z=col_z, log_z=True if col_z == "count" else False, 
                           ax=axs[ii], nbins=100, vmin=1, vmax=1e3, contours=True, colors="white", 
                           plot_colorbar=True if ii == 2 else False)

    # Decorations
    axs[1].set_title(cat)
    [ax.set_ylabel("") for ax in axs[1:]]
    [ax.set_yticklabels([]) for ax in axs[1:]]

    # Grid on
    [ax.grid() for ax in axs]

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
"""
# ///////////////////////////////////////////////////////////////////////////////
# SF-like spaxels
#///////////////////////////////////////////////////////////////////////////////
# Wind: number of components > 1, AND EITHER delta sigma of 1 or 2 is > 0
# Note: may want to also add if ncomponents == 1 but delta_sigma >> 0. 
# How many SF (either classified via BPT or N2) spaxels are there like this, though? Just checked - only ~0.1% have dsigma > 0 by 3sigma, so probably don't worry 
cond_SF_no_wind = cond_SF & ~(df_lzifu["Kinematically disturbed (component 0)"] | df_lzifu["Kinematically disturbed (component 1)"] | df_lzifu["Kinematically disturbed (component 2)"])
df_lzifu.loc[cond_SF_no_wind, "WHAV*"] = "SF + no wind"

cond_SF_wind = cond_SF & (df_lzifu["Kinematically disturbed (component 0)"] | df_lzifu["Kinematically disturbed (component 1)"] | df_lzifu["Kinematically disturbed (component 2)"])
df_lzifu.loc[cond_SF_wind, "WHAV*"] = "SF + wind"

# SF + HOLMES 
cond_SF_no_wind_HOLMES = cond_SF_no_wind & (df_lzifu["Number of components"] >= 2) & (df_lzifu["Possible HOLMES (component 0)"] | df_lzifu["Possible HOLMES (component 1)"] | df_lzifu["Possible HOLMES (component 2)"])
df_lzifu.loc[cond_SF_no_wind_HOLMES, "WHAV*"] = "SF + HOLMES + no wind"

cond_SF_wind_HOLMES = cond_SF_wind & (df_lzifu["Number of components"] >= 2) & (df_lzifu["Possible HOLMES (component 0)"] | df_lzifu["Possible HOLMES (component 1)"] | df_lzifu["Possible HOLMES (component 2)"])
df_lzifu.loc[cond_SF_wind_HOLMES, "WHAV*"] = "SF + HOLMES + wind"


# Note: what to do about low-metallicity AGN? e.g., ones that are classified as ambiguous that have log N2 < -0.35 so get lumped in with SF?

#///////////////////////////////////////////////////////////////////////////////
# Mixing-like spaxels
#///////////////////////////////////////////////////////////////////////////////
# wind/no wind
# Note: <1% of composite/mixing-like spaxels have ncomponents == 1 but delta_sigma >> 0 by 3sigma
cond_Mixing_no_wind = cond_Mixing & ~(df_lzifu["Kinematically disturbed (component 0)"] | df_lzifu["Kinematically disturbed (component 1)"] | df_lzifu["Kinematically disturbed (component 2)"])
df_lzifu.loc[cond_Mixing_no_wind, "WHAV*"] = "Mixing + no wind"

cond_Mixing_wind = cond_Mixing & (df_lzifu["Kinematically disturbed (component 0)"] | df_lzifu["Kinematically disturbed (component 1)"] | df_lzifu["Kinematically disturbed (component 2)"])
df_lzifu.loc[cond_Mixing_wind, "WHAV*"] = "Mixing + wind"

# Mixing + HOLMES 
cond_Mixing_no_wind_HOLMES = cond_Mixing_no_wind & (df_lzifu["Number of components"] >= 2) & (df_lzifu["Possible HOLMES (component 0)"] | df_lzifu["Possible HOLMES (component 1)"] | df_lzifu["Possible HOLMES (component 2)"])
df_lzifu.loc[cond_Mixing_no_wind_HOLMES, "WHAV*"] = "Mixing + HOLMES + no wind"

# Mixing + HOLMES + wind
cond_Mixing_wind_HOLMES = cond_Mixing_wind & (df_lzifu["Number of components"] >= 2) & (df_lzifu["Possible HOLMES (component 0)"] | df_lzifu["Possible HOLMES (component 1)"] | df_lzifu["Possible HOLMES (component 2)"])
df_lzifu.loc[cond_Mixing_wind_HOLMES, "WHAV*"] = "Mixing + HOLMES + wind"

#///////////////////////////////////////////////////////////////////////////////
# AGN-like spaxels
#///////////////////////////////////////////////////////////////////////////////
# If there is 1 component and its EW is > 0, then it's an AGN. Note that Seyfert-like components have a range of EWs, so we can't really split between LLAGN and Seyferts here - really need [OIII] for that.
cond_AGN_no_wind = cond_AGN & (df_lzifu["Number of components"] == 1) & (df_lzifu["HALPHA EW (component 0)"] > 3) & ~df_lzifu["Kinematically disturbed (component 0)"]
df_lzifu.loc[cond_AGN_no_wind, "WHAV*"] = "AGN only"

# AGN + wind
cond_AGN_nowind = cond_AGN & ~(df_lzifu["Kinematically disturbed (component 0)"] | df_lzifu["Kinematically disturbed (component 1)"] | (df_lzifu["Kinematically disturbed (component 2)"] ))
df_lzifu.loc[cond_AGN_nowind, "WHAV*"] = "AGN + no wind"

cond_AGN_wind = cond_AGN & (df_lzifu["Kinematically disturbed (component 0)"] | df_lzifu["Kinematically disturbed (component 1)"] | (df_lzifu["Kinematically disturbed (component 2)"] ))
df_lzifu.loc[cond_AGN_wind, "WHAV*"] = "AGN + wind"

# If there are multiple components and at least one of them is in the HOLMES regime, then classify it as HOLMES + AGN. 
cond_AGN_nowind_HOLMES = cond_AGN_nowind & (df_lzifu["Number of components"] >= 2) & (df_lzifu["Possible HOLMES (component 0)"] | df_lzifu["Possible HOLMES (component 1)"] | df_lzifu["Possible HOLMES (component 2)"])
df_lzifu.loc[cond_AGN_nowind_HOLMES, "WHAV*"] = "AGN + HOLMES + no wind"

cond_AGN_wind_HOLMES = cond_AGN_wind & (df_lzifu["Number of components"] >= 2) & (df_lzifu["Possible HOLMES (component 0)"] | df_lzifu["Possible HOLMES (component 1)"] | df_lzifu["Possible HOLMES (component 2)"])
df_lzifu.loc[cond_AGN_wind_HOLMES, "WHAV*"] = "AGN + HOLMES + wind"

#///////////////////////////////////////////////////////////////////////////////
# Numerical labels
#///////////////////////////////////////////////////////////////////////////////
num_dict = {
    "Unknown": -1,
    "HOLMES": 0,
    "Mixing + HOLMES + no wind": 1,
    "Mixing + HOLMES + wind": 2,
    "Mixing + no wind": 3,
    "Mixing + wind": 4,    
    "AGN + HOLMES + no wind": 5,
    "AGN + HOLMES + wind": 6,
    "AGN + no wind": 7,
    "AGN + wind": 8,
    "SF + HOLMES + no wind": 9,
    "SF + HOLMES + wind": 10,
    "SF + no wind": 11,
    "SF + wind": 12
}
cats = list(num_dict.keys())
for cat in cats:
    df_lzifu.loc[df_lzifu["WHAV*"] == cat, "WHAV* (numeric)"] = num_dict[cat]
"""

'\n# ///////////////////////////////////////////////////////////////////////////////\n# SF-like spaxels\n#///////////////////////////////////////////////////////////////////////////////\n# Wind: number of components > 1, AND EITHER delta sigma of 1 or 2 is > 0\n# Note: may want to also add if ncomponents == 1 but delta_sigma >> 0. \n# How many SF (either classified via BPT or N2) spaxels are there like this, though? Just checked - only ~0.1% have dsigma > 0 by 3sigma, so probably don\'t worry \ncond_SF_no_wind = cond_SF & ~(df_lzifu["Kinematically disturbed (component 0)"] | df_lzifu["Kinematically disturbed (component 1)"] | df_lzifu["Kinematically disturbed (component 2)"])\ndf_lzifu.loc[cond_SF_no_wind, "WHAV*"] = "SF + no wind"\n\ncond_SF_wind = cond_SF & (df_lzifu["Kinematically disturbed (component 0)"] | df_lzifu["Kinematically disturbed (component 1)"] | df_lzifu["Kinematically disturbed (component 2)"])\ndf_lzifu.loc[cond_SF_wind, "WHAV*"] = "SF + wind"\n\n# SF + HOLMES \ncond

### How do our classifications match up with BPT classifcations?
---

In [20]:
#############################################################################################################
# Table: comparison between BPT- and N2- and \ewha{}-based spectral categories for the LZIFU high-SN subset.
#############################################################################################################
whav_cats = list(reversed(whav_labels))
s = f"{'':25s}"
s_tot = f"{'Total':25s}"
for bpt in ["SF", "Composite", "Seyfert", "LINER", "Ambiguous", "Not classified"]:
    s += f"& {bpt:15s}"
    N_bpt_tot = df_lzifu[(df_lzifu["BPT (total)"] == bpt)].shape[0]
    s_tot += f"& {N_bpt_tot:15d} "
s += "\\\\"
s_tot += "\\\\"
print(s)
for whav_cat in whav_cats:
    s = f"{whav_cat:25s}"
    for bpt in ["SF", "Composite", "Seyfert", "LINER", "Ambiguous", "Not classified"]:
        N_bpt_tot =         df_lzifu[(df_lzifu["BPT (total)"] == bpt)].shape[0]
        N_bpt_in_this_cat = df_lzifu[(df_lzifu["BPT (total)"] == bpt) & (df_lzifu["WHAV*"] == whav_cat)].shape[0]
        s += f"\t& {N_bpt_in_this_cat / N_bpt_tot * 100:10.2f}\%"
    s += "\\\\"
    print(s)
print(s_tot)


                         & SF             & Composite      & Seyfert        & LINER          & Ambiguous      & Not classified \\
SF + wind                	&      10.73\%	&       1.80\%	&       0.09\%	&       0.00\%	&       4.27\%	&       0.26\%\\
SF + no wind             	&      83.39\%	&      19.29\%	&       1.69\%	&       0.00\%	&      66.51\%	&       4.66\%\\
SF + HOLMES + wind       	&       0.00\%	&       0.00\%	&       0.00\%	&       0.00\%	&       0.00\%	&       0.00\%\\
SF + HOLMES + no wind    	&       0.00\%	&       0.00\%	&       0.00\%	&       0.00\%	&       0.00\%	&       0.00\%\\
AGN + wind               	&       0.00\%	&       2.47\%	&      43.93\%	&       3.20\%	&       2.79\%	&       0.17\%\\
AGN + no wind            	&       0.01\%	&       7.28\%	&      19.19\%	&      20.88\%	&       6.71\%	&       1.49\%\\
AGN + HOLMES + wind      	&       0.00\%	&       0.09\%	&       2.07\%	&       1.43\%	&       0.12\%	&       0.04\%\\
AGN + HOLMES + no wind   	&       0.00\%	&  

In [21]:
#############################################################################################################
# Table: comparison between BPT- and N2- and \ewha{}-based spectral categories for the entire SAMI sample.
#############################################################################################################
whav_cats = list(reversed(whav_labels))
s = f"{'':25s}"
s_tot = f"{'Total':25s}"
for bpt in ["SF", "Composite", "Seyfert", "LINER", "Ambiguous", "Not classified"]:
    s += f"& {bpt:15s}"
    N_bpt_tot = df[(df["BPT (total)"] == bpt)].shape[0]
    s_tot += f"& {N_bpt_tot:15d} "
s += "\\\\"
s_tot += "\\\\"
print(s)
for whav_cat in whav_cats:
    s = f"{whav_cat:25s}"
    for bpt in ["SF", "Composite", "Seyfert", "LINER", "Ambiguous", "Not classified"]:
        N_bpt_tot =         df[(df["BPT (total)"] == bpt)].shape[0]
        N_bpt_in_this_cat = df[(df["BPT (total)"] == bpt) & (df["WHAV*"] == whav_cat)].shape[0]
        s += f"\t& {N_bpt_in_this_cat / N_bpt_tot * 100:10.2f}\%"
    s += "\\\\"
    print(s)
print(s_tot)


                         & SF             & Composite      & Seyfert        & LINER          & Ambiguous      & Not classified \\
SF + wind                	&       2.69\%	&       0.63\%	&       1.88\%	&       0.00\%	&       0.58\%	&       0.01\%\\
SF + no wind             	&      94.27\%	&      14.72\%	&       5.34\%	&       0.01\%	&      79.04\%	&       7.46\%\\
SF + HOLMES + wind       	&       0.00\%	&       0.00\%	&       0.00\%	&       0.00\%	&       0.00\%	&       0.00\%\\
SF + HOLMES + no wind    	&       0.00\%	&       0.00\%	&       0.00\%	&       0.00\%	&       0.00\%	&       0.00\%\\
AGN + wind               	&       0.00\%	&       2.23\%	&      27.80\%	&       4.57\%	&       1.08\%	&       0.04\%\\
AGN + no wind            	&       0.00\%	&       8.39\%	&      23.74\%	&      12.40\%	&       3.99\%	&       1.62\%\\
AGN + HOLMES + wind      	&       0.00\%	&       0.38\%	&       3.73\%	&       2.34\%	&       0.19\%	&       0.01\%\\
AGN + HOLMES + no wind   	&       0.00\%	&  

In [22]:
# What % of all BPT-classified SF spaxels are in a WHAV* category that includes SF?
print("\n############################################################################################")
print("LZIFU SUBSET: STAR-FORMING SPAXELS")
print("############################################################################################")
N_BPT = df_lzifu[(df_lzifu["BPT (total)"] == "SF")].shape[0]
N_agree = df_lzifu[(df_lzifu["BPT (total)"] == "SF") & (df_lzifu["WHAV*"].isin(["SF + no wind", "SF + wind", "SF + HOLMES + wind", "SF + HOLMES + no wind"]))].shape[0]
N_disagree = df_lzifu[(df_lzifu["BPT (total)"] == "SF") & ~(df_lzifu["WHAV*"].isin(["SF + no wind", "SF + wind", "SF + HOLMES + wind", "SF + HOLMES + no wind"]))].shape[0]
print(f"Fraction of BPT-classified SF spaxels with a WHAV* classification that agrees: {N_agree}/{N_BPT} = {N_agree / N_BPT * 100:.2f}%")
print(f"Fraction of BPT-classified SF spaxels with a WHAV* classification that disagrees: {N_disagree}/{N_BPT} = {N_disagree / N_BPT * 100:.2f}%")
print("--------------------------------------------------------------------------------------------")
print("Of all BPT-classified SF spaxels:")
N_tot = 0
for cat in cats:
    N_in_this_cat = df_lzifu[(df_lzifu["BPT (total)"] == "SF") & (df_lzifu["WHAV*"] == cat)].shape[0]
    print(f"\t{N_in_this_cat:4d}/{N_BPT:4d} = {N_in_this_cat / N_BPT * 100:3.2f}% are in category '{cat}'")
    N_tot += N_in_this_cat

# Composite/mixng?
print("\n############################################################################################")
print("LZIFU SUBSET: COMPOSITE SPAXELS")
print("############################################################################################")

N_BPT = df_lzifu[(df_lzifu["BPT (total)"] == "Composite")].shape[0]
N_agree = df_lzifu[(df_lzifu["BPT (total)"] == "Composite") & (df_lzifu["WHAV*"].isin(["Mixing + no wind", "Mixing + wind", "Mixing + HOLMES + wind", "Mixing + HOLMES + no wind"]))].shape[0]
N_disagree = df_lzifu[(df_lzifu["BPT (total)"] == "Composite") & ~(df_lzifu["WHAV*"].isin(["Mixing + no wind", "Mixing + wind", "Mixing + HOLMES + wind", "Mixing + HOLMES + no wind"]))].shape[0]
print(f"Fraction of BPT-classified composite spaxels with a WHAV* classification that agrees: {N_agree}/{N_BPT} = {N_agree / N_BPT * 100:.2f}%")
print(f"Fraction of BPT-classified composite spaxels with a WHAV* classification that disagrees: {N_disagree}/{N_BPT} = {N_disagree / N_BPT * 100:.2f}%")
print("--------------------------------------------------------------------------------------------")
print("Of all BPT-classified Composite spaxels:")
N_tot = 0
for cat in cats:
    N_in_this_cat = df_lzifu[(df_lzifu["BPT (total)"] == "Composite") & (df_lzifu["WHAV*"] == cat)].shape[0]
    print(f"\t{N_in_this_cat:4d}/{N_BPT:4d} = {N_in_this_cat / N_BPT * 100:3.2f}% are in category '{cat}'")
    N_tot += N_in_this_cat

# Seyferts
print("\n############################################################################################")
print("LZIFU SUBSET: SEYFERT SPAXELS")
print("############################################################################################")
N_BPT = df_lzifu[(df_lzifu["BPT (total)"] == "Seyfert")].shape[0]  
N_agree = df_lzifu[(df_lzifu["BPT (total)"] == "Seyfert") & (df_lzifu["WHAV*"].isin(["AGN + no wind", "AGN + wind", "AGN + HOLMES + wind", "AGN + HOLMES + no wind"]))].shape[0]
N_disagree = df_lzifu[(df_lzifu["BPT (total)"] == "Seyfert") & ~(df_lzifu["WHAV*"].isin(["AGN + no wind", "AGN + wind", "AGN + HOLMES + wind", "AGN + HOLMES + no wind"]))].shape[0]
print(f"Fraction of BPT-classified Seyfert spaxels with a WHAV* classification that agrees: {N_agree}/{N_BPT} = {N_agree / N_BPT * 100:.2f}%")
print(f"Fraction of BPT-classified Seyfert spaxels with a WHAV* classification that disagrees: {N_disagree}/{N_BPT} = {N_disagree / N_BPT * 100:.2f}%")
print("--------------------------------------------------------------------------------------------")
print("Of all BPT-classified Seyfert spaxels:")
N_tot = 0
for cat in cats:
    N_in_this_cat = df_lzifu[(df_lzifu["BPT (total)"] == "Seyfert") & (df_lzifu["WHAV*"] == cat)].shape[0]
    print(f"\t{N_in_this_cat:4d}/{N_BPT:4d} = {N_in_this_cat / N_BPT * 100:3.2f}% are in category '{cat}'")
    N_tot += N_in_this_cat

# LINERs
print("\n############################################################################################")
print("LZIFU SUBSET: LINER SPAXELS")
print("############################################################################################")
N_BPT = df_lzifu[(df_lzifu["BPT (total)"] == "LINER")].shape[0]  
N_agree = df_lzifu[(df_lzifu["BPT (total)"] == "LINER") & (df_lzifu["WHAV*"].isin(["HOLMES", "AGN + no wind", "AGN + wind", "AGN + HOLMES + wind", "AGN + HOLMES + no wind"]))].shape[0]
N_disagree = df_lzifu[(df_lzifu["BPT (total)"] == "LINER") & ~(df_lzifu["WHAV*"].isin(["HOLMES", "AGN + no wind", "AGN + wind", "AGN + HOLMES + wind", "AGN + HOLMES + no wind"]))].shape[0]
print(f"Fraction of BPT-classified LINER spaxels with a WHAV* classification that agrees: {N_agree}/{N_BPT} = {N_agree / N_BPT * 100:.2f}%")
print(f"Fraction of BPT-classified LINER spaxels with a WHAV* classification that disagrees: {N_disagree}/{N_BPT} = {N_disagree / N_BPT * 100:.2f}%")
print("--------------------------------------------------------------------------------------------")
print("Of all BPT-classified LINER spaxels:")
N_tot = 0
for cat in cats:
    N_in_this_cat = df_lzifu[(df_lzifu["BPT (total)"] == "LINER") & (df_lzifu["WHAV*"] == cat)].shape[0]
    print(f"\t{N_in_this_cat:4d}/{N_BPT:4d} = {N_in_this_cat / N_BPT * 100:3.2f}% are in category '{cat}'")
    N_tot += N_in_this_cat

    
# Ambiguous
print("\n############################################################################################")
print("LZIFU SUBSET: AMBIGUOUS SPAXELS")
print("############################################################################################")
N_BPT = df_lzifu[(df_lzifu["BPT (total)"] == "Ambiguous")].shape[0]  
print("Of all BPT-classified Ambiguous spaxels:")
N_tot = 0
for cat in cats:
    N_in_this_cat = df_lzifu[(df_lzifu["BPT (total)"] == "Ambiguous") & (df_lzifu["WHAV*"] == cat)].shape[0]
    print(f"\t{N_in_this_cat:4d}/{N_BPT:4d} = {N_in_this_cat / N_BPT * 100:3.2f}% are in category '{cat}'")
    N_tot += N_in_this_cat

# NOT CLASSIFIED
print("\n############################################################################################")
print("LZIFU SUBSET: NOT CLASSIFIED SPAXELS")
print("############################################################################################")
N_BPT = df_lzifu[(df_lzifu["BPT (total)"] == "Not classified")].shape[0]  
print("Of all BPT-classified Not classified spaxels:")
N_tot = 0
for cat in cats:
    N_in_this_cat = df_lzifu[(df_lzifu["BPT (total)"] == "Not classified") & (df_lzifu["WHAV*"] == cat)].shape[0]
    print(f"\t{N_in_this_cat:4d}/{N_BPT:4d} = {N_in_this_cat / N_BPT * 100:3.2f}% are in category '{cat}'")
    N_tot += N_in_this_cat
    
# Finally: how many spaxels do have Halpha but don't have an N2 measurement?
print("\n############################################################################################")
print("LZIFU SUBSET: SPAXELS WITH NO N2")
print("############################################################################################")
N_tot = df_lzifu[~np.isnan(df_lzifu["HALPHA EW (component 0)"]) & ~np.isnan(df_lzifu["sigma_gas - sigma_* (component 0)"])].shape[0]
N_no_N2 = df_lzifu[(df_lzifu["WHAV*"] == "No N2") & (~np.isnan(df_lzifu["HALPHA EW (component 0)"]) & ~np.isnan(df_lzifu["sigma_gas - sigma_* (component 0)"]))].shape[0]
print(f"{N_no_N2:4d}/{N_tot:4d} = {N_no_N2 / N_tot * 100:3.2f}% of spaxels with EW and delta_sigma measurements have no N2 measurement")


############################################################################################
LZIFU SUBSET: STAR-FORMING SPAXELS
############################################################################################
Fraction of BPT-classified SF spaxels with a WHAV* classification that agrees: 25733/27339 = 94.13%
Fraction of BPT-classified SF spaxels with a WHAV* classification that disagrees: 1606/27339 = 5.87%
--------------------------------------------------------------------------------------------
Of all BPT-classified SF spaxels:


NameError: name 'cats' is not defined

In [ ]:
# What % of all BPT-classified SF spaxels are in a WHAV* category that includes SF?
print("\n############################################################################################")
print("ENTIRE SAMI SAMPLE: STAR-FORMING SPAXELS")
print("############################################################################################")
N_BPT = df[(df["BPT (total)"] == "SF")].shape[0]
N_agree = df[(df["BPT (total)"] == "SF") & (df["WHAV*"].isin(["SF + no wind", "SF + wind", "SF + HOLMES + wind", "SF + HOLMES + no wind"]))].shape[0]
N_disagree = df[(df["BPT (total)"] == "SF") & ~(df["WHAV*"].isin(["SF + no wind", "SF + wind", "SF + HOLMES + wind", "SF + HOLMES + no wind"]))].shape[0]
print(f"Fraction of BPT-classified SF spaxels with a WHAV* classification that agrees: {N_agree}/{N_BPT} = {N_agree / N_BPT * 100:.2f}%")
print(f"Fraction of BPT-classified SF spaxels with a WHAV* classification that disagrees: {N_disagree}/{N_BPT} = {N_disagree / N_BPT * 100:.2f}%")
print("--------------------------------------------------------------------------------------------")
print("Of all BPT-classified SF spaxels:")
N_tot = 0
for cat in cats:
    N_in_this_cat = df[(df["BPT (total)"] == "SF") & (df["WHAV*"] == cat)].shape[0]
    print(f"\t{N_in_this_cat:4d}/{N_BPT:4d} = {N_in_this_cat / N_BPT * 100:3.2f}% are in category '{cat}'")
    N_tot += N_in_this_cat

# Composite/mixng?
print("\n############################################################################################")
print("ENTIRE SAMI SAMPLE: COMPOSITE SPAXELS")
print("############################################################################################")

N_BPT = df[(df["BPT (total)"] == "Composite")].shape[0]
N_agree = df[(df["BPT (total)"] == "Composite") & (df["WHAV*"].isin(["Mixing + no wind", "Mixing + wind", "Mixing + HOLMES + wind", "Mixing + HOLMES + no wind"]))].shape[0]
N_disagree = df[(df["BPT (total)"] == "Composite") & ~(df["WHAV*"].isin(["Mixing + no wind", "Mixing + wind", "Mixing + HOLMES + wind", "Mixing + HOLMES + no wind"]))].shape[0]
print(f"Fraction of BPT-classified composite spaxels with a WHAV* classification that agrees: {N_agree}/{N_BPT} = {N_agree / N_BPT * 100:.2f}%")
print(f"Fraction of BPT-classified composite spaxels with a WHAV* classification that disagrees: {N_disagree}/{N_BPT} = {N_disagree / N_BPT * 100:.2f}%")
print("--------------------------------------------------------------------------------------------")
print("Of all BPT-classified Composite spaxels:")
N_tot = 0
for cat in cats:
    N_in_this_cat = df[(df["BPT (total)"] == "Composite") & (df["WHAV*"] == cat)].shape[0]
    print(f"\t{N_in_this_cat:4d}/{N_BPT:4d} = {N_in_this_cat / N_BPT * 100:3.2f}% are in category '{cat}'")
    N_tot += N_in_this_cat

# Seyferts
print("\n############################################################################################")
print("ENTIRE SAMI SAMPLE: SEYFERT SPAXELS")
print("############################################################################################")
N_BPT = df[(df["BPT (total)"] == "Seyfert")].shape[0]  
N_agree = df[(df["BPT (total)"] == "Seyfert") & (df["WHAV*"].isin(["AGN + no wind", "AGN + wind", "AGN + HOLMES + wind", "AGN + HOLMES + no wind"]))].shape[0]
N_disagree = df[(df["BPT (total)"] == "Seyfert") & ~(df["WHAV*"].isin(["AGN + no wind", "AGN + wind", "AGN + HOLMES + wind", "AGN + HOLMES + no wind"]))].shape[0]
print(f"Fraction of BPT-classified Seyfert spaxels with a WHAV* classification that agrees: {N_agree}/{N_BPT} = {N_agree / N_BPT * 100:.2f}%")
print(f"Fraction of BPT-classified Seyfert spaxels with a WHAV* classification that disagrees: {N_disagree}/{N_BPT} = {N_disagree / N_BPT * 100:.2f}%")
print("--------------------------------------------------------------------------------------------")
print("Of all BPT-classified Seyfert spaxels:")
N_tot = 0
for cat in cats:
    N_in_this_cat = df[(df["BPT (total)"] == "Seyfert") & (df["WHAV*"] == cat)].shape[0]
    print(f"\t{N_in_this_cat:4d}/{N_BPT:4d} = {N_in_this_cat / N_BPT * 100:3.2f}% are in category '{cat}'")
    N_tot += N_in_this_cat

# LINERs
print("\n############################################################################################")
print("ENTIRE SAMI SAMPLE: LINER SPAXELS")
print("############################################################################################")
N_BPT = df[(df["BPT (total)"] == "LINER")].shape[0]  
N_agree = df[(df["BPT (total)"] == "LINER") & (df["WHAV*"].isin(["HOLMES", "AGN + no wind", "AGN + wind", "AGN + HOLMES + wind", "AGN + HOLMES + no wind"]))].shape[0]
N_disagree = df[(df["BPT (total)"] == "LINER") & ~(df["WHAV*"].isin(["HOLMES", "AGN + no wind", "AGN + wind", "AGN + HOLMES + wind", "AGN + HOLMES + no wind"]))].shape[0]
print(f"Fraction of BPT-classified LINER spaxels with a WHAV* classification that agrees: {N_agree}/{N_BPT} = {N_agree / N_BPT * 100:.2f}%")
print(f"Fraction of BPT-classified LINER spaxels with a WHAV* classification that disagrees: {N_disagree}/{N_BPT} = {N_disagree / N_BPT * 100:.2f}%")
print("--------------------------------------------------------------------------------------------")
print("Of all BPT-classified LINER spaxels:")
N_tot = 0
for cat in cats:
    N_in_this_cat = df[(df["BPT (total)"] == "LINER") & (df["WHAV*"] == cat)].shape[0]
    print(f"\t{N_in_this_cat:4d}/{N_BPT:4d} = {N_in_this_cat / N_BPT * 100:3.2f}% are in category '{cat}'")
    N_tot += N_in_this_cat

    
# Ambiguous
print("\n############################################################################################")
print("ENTIRE SAMI SAMPLE: AMBIGUOUS SPAXELS")
print("############################################################################################")
N_BPT = df[(df["BPT (total)"] == "Ambiguous")].shape[0]  
print("Of all BPT-classified Ambiguous spaxels:")
N_tot = 0
for cat in cats:
    N_in_this_cat = df[(df["BPT (total)"] == "Ambiguous") & (df["WHAV*"] == cat)].shape[0]
    print(f"\t{N_in_this_cat:4d}/{N_BPT:4d} = {N_in_this_cat / N_BPT * 100:3.2f}% are in category '{cat}'")
    N_tot += N_in_this_cat

# NOT CLASSIFIED
print("\n############################################################################################")
print("ENTIRE SAMI SAMPLE: NOT CLASSIFIED SPAXELS")
print("############################################################################################")
N_BPT = df[(df["BPT (total)"] == "Not classified")].shape[0]  
print("Of all BPT-classified Not classified spaxels:")
N_tot = 0
for cat in cats:
    N_in_this_cat = df[(df["BPT (total)"] == "Not classified") & (df["WHAV*"] == cat)].shape[0]
    print(f"\t{N_in_this_cat:4d}/{N_BPT:4d} = {N_in_this_cat / N_BPT * 100:3.2f}% are in category '{cat}'")
    N_tot += N_in_this_cat
    
# Finally: how many spaxels do have Halpha but don't have an N2 measurement?
print("\n############################################################################################")
print("ENTIRE SAMI SAMPLE: SPAXELS WITH NO N2")
print("############################################################################################")
N_tot = df[~np.isnan(df["HALPHA EW (component 0)"]) & ~np.isnan(df["sigma_gas - sigma_* (component 0)"])].shape[0]
N_no_N2 = df[(df["WHAV*"] == "No N2") & (~np.isnan(df["HALPHA EW (component 0)"]) & ~np.isnan(df["sigma_gas - sigma_* (component 0)"]))].shape[0]
print(f"{N_no_N2:4d}/{N_tot:4d} = {N_no_N2 / N_tot * 100:3.2f}% of spaxels with EW and delta_sigma measurements have no N2 measurement")


### BPT, WHAN, WHAV* diagrams to compare classifications
---

In [ ]:
#///////////////////////////////////////////////////////////////////////////////
# BPT (based on total fluxes): all categories
col_z = "WHAV* (numeric)"
fig, axs, cax = plot_empty_BPT_diagram(colorbar=True, nrows=1, include_Law2021=True)

# Plot 2D histograms of the subset
plot2dhistcontours(df, col_x="log N2 (total)", col_y="log O3 (total)", col_z=col_z, log_z=True if col_z == "count" else False, ax=axs[0], nbins=100, contours=True, colors="white", plot_colorbar=False)
plot2dhistcontours(df, col_x="log S2 (total)", col_y="log O3 (total)", col_z=col_z, log_z=True if col_z == "count" else False, ax=axs[1], nbins=100, contours=True, colors="white", plot_colorbar=False)
plot2dhistcontours(df, col_x="log O1 (total)", col_y="log O3 (total)", col_z=col_z, log_z=True if col_z == "count" else False, ax=axs[2], nbins=100, contours=True, colors="white", cax=cax, plot_colorbar=True)

# Decorations
# axs[1].set_title(cat)
[ax.set_ylabel("") for ax in axs[1:]]

# Grid on
[ax.grid() for ax in axs]

In [ ]:
#///////////////////////////////////////////////////////////////////////////////
# BPT (based on total fluxes): each category shown separately
col_z = "WHAV* (numeric)"
for cat in df_lzifu["WHAV*"].unique()[1:2]:
    df_cat = df_lzifu[df_lzifu["WHAV*"] == cat]
    if df_cat.shape[0] == 0:
        continue
    fig, axs, cax = plot_empty_BPT_diagram(colorbar=True, nrows=1, include_Law2021=True)

    # Plot 2D histograms of the subset
    plot2dhistcontours(df_cat, col_x="log N2 (total)", col_y="log O3 (total)", col_z=col_z, log_z=True if col_z == "count" else False, ax=axs[0], nbins=100, contours=True, colors="white", plot_colorbar=False)
    plot2dhistcontours(df_cat, col_x="log S2 (total)", col_y="log O3 (total)", col_z=col_z, log_z=True if col_z == "count" else False, ax=axs[1], nbins=100, contours=True, colors="white", plot_colorbar=False)
    plot2dhistcontours(df_cat, col_x="log O1 (total)", col_y="log O3 (total)", col_z=col_z, log_z=True if col_z == "count" else False, ax=axs[2], nbins=100, contours=True, colors="white", cax=cax, plot_colorbar=True)

    # Decorations
    axs[1].set_title(cat)
    [ax.set_ylabel("") for ax in axs[1:]]

    # Grid on
    [ax.grid() for ax in axs]

In [ ]:
plt.close("all")

In [ ]:
#///////////////////////////////////////////////////////////////////////////////
# WHAN: all categories at once 
col_x = "log N2"
col_y = "log HALPHA EW"
col_z = "WHAV* (numeric)"

fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(15, 4))
fig.subplots_adjust(wspace=0)
bbox = axs[-1].get_position()
fig.add_axes([bbox.x0 + bbox.width, bbox.y0, bbox.width * 0.1, bbox.height])

for ii in range(3):
    if all(df_cat[f"{col_x} (component {ii})"].isna()) or all(df_cat[f"{col_y} (component {ii})"].isna()):
        continue
    plot2dhistcontours(df_lzifu, 
                       col_x=f"{col_x} (component {ii})", 
                       col_y=f"{col_y} (component {ii})",
                       col_z=col_z, log_z=True if col_z == "count" else False, 
                       ax=axs[ii], nbins=100, contours=True, colors="white", 
                       plot_colorbar=True if ii == 2 else False)

# Decorations
[ax.set_ylabel("") for ax in axs[1:]]
[ax.set_yticklabels([]) for ax in axs[1:]]

# Grid on
[ax.grid() for ax in axs]

In [ ]:
#///////////////////////////////////////////////////////////////////////////////
# WHAN: each category separately
col_x = "log N2"
col_y = "log HALPHA EW"
col_z = "WHAV* (numeric)"

for cat in df_lzifu["WHAV*"].unique()[1:]:
    df_cat = df_lzifu[df_lzifu["WHAV*"] == cat]
    if df_cat.shape[0] == 0:
        continue
        
    fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(15, 4))
    fig.subplots_adjust(wspace=0)
    bbox = axs[-1].get_position()
    fig.add_axes([bbox.x0 + bbox.width, bbox.y0, bbox.width * 0.1, bbox.height])
    
    for ii in range(3):
        if all(df_cat[f"{col_x} (component {ii})"].isna()) or all(df_cat[f"{col_y} (component {ii})"].isna()):
            continue
        plot2dhistcontours(df_cat, 
                           col_x=f"{col_x} (component {ii})", 
                           col_y=f"{col_y} (component {ii})",
                           col_z=col_z, log_z=True if col_z == "count" else False, 
                           ax=axs[ii], nbins=100, contours=True, colors="white", 
                           plot_colorbar=True if ii == 2 else False)

    # Decorations
    axs[1].set_title(cat)
    [ax.set_ylabel("") for ax in axs[1:]]
    [ax.set_yticklabels([]) for ax in axs[1:]]

    # Grid on
    [ax.grid() for ax in axs]

In [ ]:
df_lzifu[df_lzifu["WHAV*"] == "SF + no wind"]

In [ ]:
#///////////////////////////////////////////////////////////////////////////////
# WHAV*: all categories at once 
col_x = "sigma_gas - sigma_*"
col_y = "log HALPHA EW"
col_z = "WHAV* (numeric)"

fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(15, 4))
fig.subplots_adjust(wspace=0)
bbox = axs[-1].get_position()
fig.add_axes([bbox.x0 + bbox.width, bbox.y0, bbox.width * 0.1, bbox.height])

for ii in range(3):
    plot2dhistcontours(df_lzifu, 
                       col_x=f"{col_x} (component {ii})", 
                       col_y=f"{col_y} (component {ii})",
                       col_z=col_z, log_z=True if col_z == "count" else False, 
                       ax=axs[ii], nbins=100, contours=True, colors="white", 
                       plot_colorbar=True if ii == 2 else False)

# Decorations
[ax.set_ylabel("") for ax in axs[1:]]
[ax.set_yticklabels([]) for ax in axs[1:]]

# Grid on
[ax.grid() for ax in axs]

In [ ]:
#///////////////////////////////////////////////////////////////////////////////
# WHAV*: each category shown separately
col_x = "sigma_gas - sigma_*"
col_y = "log HALPHA EW"
col_z = "WHAV* (numeric)"

for cat in df_lzifu["WHAV*"].unique()[1:]:
    df_cat = df_lzifu[df_lzifu["WHAV*"] == cat]
    if df_cat.shape[0] == 0:
        continue
        
    fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(15, 4))
    fig.subplots_adjust(wspace=0)
    bbox = axs[-1].get_position()
    fig.add_axes([bbox.x0 + bbox.width, bbox.y0, bbox.width * 0.1, bbox.height])
    
    for ii in range(3):
        if all(df_cat[f"{col_x} (component {ii})"].isna()) or all(df_cat[f"{col_y} (component {ii})"].isna()):
            continue
        plot2dhistcontours(df_cat, 
                           col_x=f"{col_x} (component {ii})", 
                           col_y=f"{col_y} (component {ii})",
                           col_z=col_z, log_z=True if col_z == "count" else False, 
                           ax=axs[ii], nbins=100, contours=True, colors="white", 
                           plot_colorbar=True if ii == 2 else False)

    # Decorations
    axs[1].set_title(cat)
    [ax.set_ylabel("") for ax in axs[1:]]
    [ax.set_yticklabels([]) for ax in axs[1:]]

    # Grid on
    [ax.grid() for ax in axs]

## Check our classification system: look at galaxies
---

Plot 2D maps showing:
* WHAV* classification
* BPT classification (total)
* Total HALPHA EW 
* Number of components

In [ ]:
df_lzifu.catid

In [ ]:
gal = 487027
df_gal = df_lzifu[df_lzifu.catid == gal]

In [ ]:
plt.close("all")

In [ ]:
gals = []

# Plot an SDSS image too 
_, ax = plt.subplots(1, 1, figsize=(5, 5))
plot_sdss_image(df_gal, ax=ax)

rc("font",**{"family": "serif", "size": 8})

# Plot 2D maps showing each quantity across the galaxy
col_z_list = "WHAV* (numeric)", "BPT (numeric) (total)", "HALPHA EW (total)", "Number of components"
fig, axs = plt.subplots(2, 2, squeeze=True, figsize=(18, 10))
for cc, col_z in enumerate(col_z_list):
    # Plot the number of components fitted.
    plot2dmap(df_gal=df_gal, bin_type="default", survey="sami",
              PA_deg=0,
              col_z=col_z,
              ax=axs.flat[cc], plot_colorbar=True, cax_orientation="vertical", show_title=False)

# Reset font size
rc("font",**{"family": "serif", "size": 14})

In [ ]:
plt.close("all")